# Distance between ants
Function to calculate the distance between specific ants (e.g., focal and caregiver ants) from a ```.mymridon``` experiment file. <br><br>
There is probably an easier way to do this by querying individual frames directly.

In [ ]:
from datetime import datetime, timedelta  # For convenient handling of time and date

import numpy as np
import pandas as pd  # Used to create a dataframe, similar to the structure used in R
import py_fort_myrmidon as fm

### Function to obtain trajectories of ants that match a specific matcher query

In [ ]:
def trajectory_output(start_time, end_time, exp, matcher_query):
    """
    Function to extract daily trajectories as a parquet file, grouped by AntID. While it is setup to extract daily trajectories, it can work for any arbitrary time duration
    :param start_time: The start datetime object. this will be converted to a fort-myrmidon Time object
    :param end_time: The end datetime object. this will be converted to a fort-myrmidon Time object
    :param exp: The name of the experiment i.e., the myrmidon file
    :param matcher_query: The fm matcher corresponding to the focal IDs
    :return: Outputs a pandas dataframe containing AntID, Space, Time, X_coordinates and Y_coordinates of each ID averaged over 1 second from the X and Y coordinates. Averagingg is done to have a dataset which can be merged across IDs using at the resolution of 1s.
    """
    start = datetime.now()
    t_begin = fm.Time(start_time)
    t_stop = fm.Time(end_time)
    trajectory = fm.Query.ComputeAntTrajectories(
        experiment=exp,
        start=t_begin,
        end=t_stop,
        matcher=matcher_query,
        maximumGap=fm.Duration.Parse("1000h"),
    )
    # Make a list of lists with trajectory values needed. Position is an array of 5 columns, so specific columns are called
    traj_list = [
        [
            trajectory.Ant,
            trajectory.Space,
            trajectory.Start.ToDateTime(),
            trajectory.Positions[:, 0],
            trajectory.Positions[:, 1],
            trajectory.Positions[:, 2],
        ]
        for trajectory in trajectory
    ]
    # Make the list into a dataframe
    traj_df = pd.DataFrame(
        traj_list,
        columns=["AntID", "Space", "StartTime", "Pos_time", "X_coor", "Y_coor"],
    )
    # Explode columns which are in the form of lists to expand the dataframe
    traj_df = traj_df.explode(column=["Pos_time", "X_coor", "Y_coor"])
    # Coerce coordinates to integer
    traj_df["X_coor"] = pd.to_numeric(traj_df["X_coor"], errors="coerce")
    traj_df["Y_coor"] = pd.to_numeric(traj_df["Y_coor"], errors="coerce")
    # Convert Pos_time to timedelta and obtain actual datetime for each trajectory entry
    traj_df["Pos_time"] = pd.to_numeric(traj_df["Pos_time"], errors="coerce")
    traj_df["Pos_time"] = pd.to_timedelta(
        traj_df["Pos_time"], unit="S", errors="coerce"
    )
    traj_df["Time"] = traj_df["StartTime"] + traj_df["Pos_time"]
    # Drop unwanted ccolumns
    traj_df = traj_df.drop(["StartTime", "Pos_time"], axis=1)
    # Reorder columns
    traj_df = traj_df[["AntID", "Space", "Time", "X_coor", "Y_coor"]]
    if traj_df.empty:  # If no trajectories are output
        # empty_row = pd.DataFrame([{'AntID': 'Unknown', 'Space':np.nan, 'Time':np.nan, 'X_coor':np.nan, 'Y_coor':np.nan}]) # Create empty row with unknown as antID
        # traj_df = pd.concat([empty_row]) # Add empty row to dataframe
        print("No trajectories found. Created empty dataframe")
        return traj_df  # Return empty dataframe
    # Obtain average X and Y coordinates per second
    traj_df = (
        traj_df.groupby([pd.Grouper(key="Time", freq="1s"), "AntID", "Space"])
        .agg(X_mean=("X_coor", "mean"), Y_mean=("Y_coor", "mean"))
        .reset_index()
    )
    end = datetime.now()
    # print("Trajectories output in", end-start)
    return traj_df

### Function to obtain displacement between focal ants and other ants

In [ ]:
def focal_caregivers_disp(start_time, end_time, exp, focal_ID, caregiver_IDs, exp_day):
    """
    Function to obtain trajectories for focal and caregiver antIDs, merge by time and calculate displacement of each caregiver ID from the focal ID at every second
    :param start_time: Starting time to obtain trajectories from. Passed on to function trajectory_output
    :param end_time: Ending time to obtain trajectories from. Passed on to function trajectory_output
    :param exp: Location of myrmidon file
    :param focal_ID: Injured AntID
    :param caregiver_IDs: IDs of caregiver ants as a list
    :param exp_day: Day of the experiment. This is added to the dataframe for identification
    :return: Returns a datafarme containing the Time (in bins of 1s based on function trajectory_output), the focal and caregiver ID, the space in which the focal and caregiver ants are present, and the displacement between them (calculated as np.nan if they are in different spaces. In a CSV output this will be converted to a blank entry).
    """
    start = datetime.now()
    # Focal Ant matcher
    focal_matcher = fm.Matcher.AntID(focal_ID)
    # Caregiver individual matchers
    caregivers = [fm.Matcher.AntID(x) for x in caregiver_IDs]
    # Create single matcher object by unpacking the list within an Or Matcher
    caregiver_matcher = fm.Matcher.Or(*caregivers)
    # Focal Ant trajectory
    focal_traj = trajectory_output(start_time, end_time, exp, focal_matcher)
    # Caregiver ants trajectories
    care_traj = trajectory_output(start_time, end_time, exp, caregiver_matcher)
    # Merge focal and caregiver trajectories on Time column
    if focal_traj.empty:  # If focal trajectory is an empty dataframe
        full_traj = care_traj.rename(
            columns={"AntID": "caregiverID", "Space": "Space_care"}
        )
        full_traj["focalID"] = full_traj["Space_focal"] = full_traj["disp"] = (
            np.nan
        )  # Create columns with na values
        full_traj = full_traj[
            ["Time", "focalID", "caregiverID", "disp", "Space_focal", "Space_care"]
        ]
        full_traj["exp_day"] = exp_day
        print(
            f"{'Focal ID trajectory is empty for list item '}{exp_day}{'.Returning dataframe with no displacement calculated'}"
        )
        return full_traj
    if care_traj.empty:  # If caregivers trajectory is an empty dataframe
        full_traj = focal_traj.rename(
            columns={"AntID": "focalID", "Space": "Space_focal"}
        )
        full_traj["caregiverID"] = full_traj["Space_care"] = full_traj["disp"] = (
            np.nan
        )  # Create columns with na values
        full_traj = full_traj[
            ["Time", "focalID", "caregiverID", "disp", "Space_focal", "Space_care"]
        ]
        full_traj["exp_day"] = exp_day
        print(
            f"{'Caregiver ID trajectories are empty for list item '}{exp_day}{'.Returning dataframe with no displacement calculated'}"
        )
        return full_traj
    full_traj = pd.merge(
        care_traj, focal_traj, how="outer", on="Time", suffixes=("_care", "_focal")
    )
    # Obtain X coordinate and Y coordinate difference between Focal and Caregivers, for each row
    full_traj["X_diff"] = full_traj["X_mean_focal"] - full_traj["X_mean_care"]
    full_traj["Y_diff"] = full_traj["Y_mean_focal"] - full_traj["Y_mean_care"]
    # Obtain displacement
    full_traj["disp"] = np.linalg.norm(
        full_traj[["X_diff", "Y_diff"]].to_numpy(), axis=1
    )
    full_traj = full_traj.rename(
        columns={"AntID_focal": "focalID", "AntID_care": "caregiverID"}
    )
    full_traj = full_traj[
        ["Time", "focalID", "caregiverID", "disp", "Space_focal", "Space_care"]
    ]
    full_traj["exp_day"] = exp_day
    # Replace with np.nan value if focal ant and caregiver are in different spaces. Use notnull to filter out instances where focal or caregiver space is not known
    full_traj.loc[
        (
            (full_traj.Space_focal.notnull())
            & (full_traj.Space_care.notnull())
            & (full_traj.Space_focal != full_traj.Space_care)
        ),
        "disp",
    ] = 0
    end = datetime.now()
    # print("Displacement calculated in", end-start)
    return full_traj

### Wrapper function to run `focal_caregivers_disp` over a whole experiment

In [ ]:
def focal_caregivers_disp_exp(disp_list, colonyID):
    """
    Helper function to run focal_caregivers_disp over multiple days in an experiment.
    :param disp_list: List containing as many elements as the number of experimental days. Each element should contain all the arguments required for focal_caregivers_disp function, including start_time, end_time, exp, focalID, caregiversID, exp_day
    :param colonyID: ColonyID to add to the dataframe for identification
    :return: Pandas dataframe containing the displacement of caregivers from the focal ant for the different experimental days in one colony
    """
    start = datetime.now()
    # Run function focal_caregivers_disp over each element in the list
    disp_df_list = [
        focal_caregivers_disp(start, end, exp, focalID, caregiversID, exp_day)
        for start, end, exp, focalID, caregiversID, exp_day in disp_list
    ]
    # Concatenate to combine data from all days
    final_df = pd.concat(disp_df_list)
    # Add colonyID
    final_df["colony"] = colonyID
    # Reorder columns
    final_df = final_df[
        [
            "colony",
            "exp_day",
            "Time",
            "focalID",
            "caregiverID",
            "disp",
            "Space_focal",
            "Space_care",
        ]
    ]
    # Sort based on colonyID, exp_day, caregiverID and time and reset index
    final_df = final_df.sort_values(
        by=["colony", "exp_day", "caregiverID", "Time"]
    ).reset_index(drop=True)
    end = datetime.now()
    print("Displacement calculated in", end - start)
    return final_df